Кобзарь Олег

Выгрузка и построение данных со СУ

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import plotly.plotly as py
import plotly.graph_objs as go
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
init_notebook_mode(connected=True)

from datetime import date

Загрузка csv без заголовков

In [ ]:
csv_file_name = "kobzar1.csv"

In [ ]:
cs_data = pd.read_csv(csv_file_name,sep=';', header=None)

In [ ]:
cs_data.head()

In [ ]:
cs_data.shape

Лист с названиями параметрами

In [ ]:
list_parametrs = np.unique(cs_data[1])

Ключ для параметра

In [ ]:
list_parametrs[0]

Число параметров

In [ ]:
len(list_parametrs)

Номер для каждого параметра и количество записей

In [ ]:
def get_information(cs_data, list_parametrs):
    for i in range(len(list_parametrs)):
        amount_of_parametr_values = len(cs_data[cs_data[1] == list_parametrs[i]])
        print("№" + str (i)+ " Кол-во записей: " + str(amount_of_parametr_values) +'  ' +str(list_parametrs[i]))

In [ ]:
get_information(cs_data, list_parametrs)

Создание DataFrame только для мощности

In [ ]:
active_power = (cs_data[cs_data[1] == list_parametrs[0]])

In [ ]:
active_power[-200:-1]

Данные по активной мощности за весь доступный период

In [ ]:
# Create a trace
trace = go.Scattergl(
    x = active_power[2].values,
    y = active_power[3].values,
    mode = 'markers'
)

data = [trace]

# Plot and embed in ipython notebook!
iplot(data, filename='basic-scatter')

In [ ]:
# Create a trace
number = 7
specific_param_by_number = (cs_data[cs_data[1] == list_parametrs[number]])
print(list_parametrs[number])
trace = go.Scattergl(
    x = specific_param_by_number[2].values,
    y = specific_param_by_number[3].values,
    mode = 'markers'
)

data = [trace]

# Plot and embed in ipython notebook!
iplot(data, filename='basic-scatter')

Функция для выбора параметра, ресемплирования и построения графиков


In [ ]:
#TODO добавить конец периода

def plot_result(number, year = 2019, month = 2, day = 1):
    
    name_param = list_parametrs[number]

    date_from = pd.Timestamp(date(year,month,day))

    data_one_parametr = (cs_data[cs_data[1] == name_param])
    data_one_parametr_resample = data_one_parametr.copy()
    data_one_parametr_resample[2] = pd.to_datetime(data_one_parametr_resample[2])
    data_one_parametr_resample = data_one_parametr_resample[data_one_parametr_resample[2]>date_from]
    data_one_parametr_resample['date'] = pd.to_datetime(data_one_parametr_resample[2], errors='coerce')
    data_one_parametr_resample.set_index = data_one_parametr_resample['date']
    data_one_parametr_resample.index = data_one_parametr_resample['date']
    data_one_parametr_resample_2 = data_one_parametr_resample.resample('1d').mean()

    trace1 = go.Scattergl(
        x = data_one_parametr_resample_2.index,
        y = data_one_parametr_resample_2[3],
        name = name_param,
        mode = 'markers'
    )

    data = [trace1]

    layout = dict(title = name_param
                 )

    fig = dict(data=data, layout=layout)

    iplot(fig, filename='basic-scatter')

In [ ]:
plot_result(38)

Функция для выделение куска данных с какого-либо времени, ресеплирования для каждого параметра и выдачи значения на последний день

In [ ]:

def print_result(number = 59, year_start = 2019, month_start = 2, day_start = 1,
                year_end = 2019, month_end = 2, day_end = 28):
    date_from = pd.Timestamp(date(year_start,month_start,day_start))
    date_end = pd.Timestamp(date(year_end,month_end,day_end))
    print('date_from ' + str(date_from))
    print('date_end ' + str(date_end))
    for i in range(number):
        
        name_param = list_parametrs[i]

        data_one_parametr = (cs_data[cs_data[1] == name_param])
        
        data_one_parametr_resample = data_one_parametr.copy()
        data_one_parametr_resample[2] = pd.to_datetime(data_one_parametr_resample[2])
        data_one_parametr_resample = data_one_parametr_resample[data_one_parametr_resample[2]>date_from]
        data_one_parametr_resample['date'] = pd.to_datetime(data_one_parametr_resample[2], errors='coerce')
        data_one_parametr_resample.set_index = data_one_parametr_resample['date']
        data_one_parametr_resample.index = data_one_parametr_resample['date']
        data_one_parametr_resample_2 = data_one_parametr_resample.resample('1d').mean()
        if data_one_parametr_resample_2.shape[0] != 0:
            value = data_one_parametr_resample_2[3][-1]
            print(str(i) + '  ' + str(name_param) + '  '  + format(value, '.2f'))


    

Результаты на последний день по данным

In [ ]:
print_result()

In [ ]:
def initial_editing(data):
    del data[0]
    str_to_delete = "КУСТ 18. ГЗУ 18А. Скважина 1354."
    data[1] = data[1].str.replace(str_to_delete,"")
    data.index = data[2]
    del data[2]
    data.index = pd.to_datetime(data.index)
    return data

def extract_df_one_parametr_and_edit(data, list_of_params, number_of_param_in_list):
    extracted_df_one_param = data[data[1] == list_of_params[number_of_param_in_list]].copy()
    edited_df_one_param = extracted_df_one_param.rename(index=str, columns = {3: extracted_df_one_param[1][1]})
    del edited_df_one_param[1]
    return edited_df_one_param

def create_edited_df(data, parametrs_list):
    init_one_parametr_df = extract_df_one_parametr_and_edit(data, parametrs_list, 0)
    result = init_one_parametr_df
    for i in range(1, len(parametrs_list)):
        new_one_parametr_df = extract_df_one_parametr_and_edit(data, parametrs_list, i)
        result = result.join([new_one_parametr_df], how = "outer", sort=True)
    return result

In [ ]:
cs_data_try_merge = cs_data[-1000:].copy()
cs_data_try_merge = initial_editing(cs_data_try_merge)
cs_data_try_merge.head()

In [ ]:
year_start = 2019
month_start = 2
day_start = 28
hour_start = 17
date_from = pd.to_datetime(pd.DataFrame({'year': [year_start],
                           'month': [month_start],
                           'day': [day_start],
                            'hour': [hour_start]}))
date_from.iloc[0]

In [ ]:
small_list_parametrs = np.unique(cs_data_try_merge[1])
get_information(cs_data_try_merge, small_list_parametrs)

In [ ]:
#small_motor_load[small_motor_load.index > date_from.iloc[0]]

In [ ]:
result_by_func = create_edited_df(cs_data_try_merge, small_list_parametrs)

In [ ]:
result_by_func.head()

In [ ]:
result_by_func.to_excel("check2.xlsx")

In [ ]:
cs_big_data_try_merge = cs_data.copy()

In [ ]:
cs_big_data_try_merge = initial_editing(cs_big_data_try_merge)

In [ ]:
big_list_parametrs = np.unique(cs_big_data_try_merge[1])

In [ ]:
big_result_by_func = create_edited_df(cs_big_data_try_merge, big_list_parametrs)

In [ ]:
#big_result_by_func.to_excel("edited_kobzar1.xlsx")

In [ ]:
big_result_by_func.to_csv("edited_kobzar1.csv")

In [ ]:
test = pd.read_csv("edited_kobzar1.csv")

In [ ]:
test.head()